# Intro To Nmap

#### Pre-requisites
- Kali Linux
- Network Lab

#### 1. Topics/Sub-Topics
** Run all demos in Kali if possible **

##### What is the goal of recon?

** INFORMATION GATHERING **

Reconnaissance is to collect as much as information about a target network as possible. From a hacker’s perspective, the information gathered is very helpful to make an attack, so to block that type of malicious attempt, generally a penetration tester tries to find the information and to patch the vulnerabilities, if found. 

NOTE: This process is also known as **footprinting**.

##### Scanning Methodologies(CEH)
** NOTE ** These are guidelines, not a rigid policy.

1. Check for live systems
2. Check for open ports
3. Scan beyond IDS
4. Perform banner grabbing
5. Scan for vulnerabilities
6. Draw network diagrams
7. Prepare proxies

What kind of things should you look for during recon?
- IP Addresses
- DNS information
- Email Address
- Ports/Protocols
- Services
- Firewall identification and evasion
- ....

##### Why nmap?
- Runs on all OS
- Network discovery
- Vulnerability detection
- Backdoor detection
- Identify services
- Inventory management
- Monitoring host/service uptime
- Customizable advanced usage with Nmap Scripting Engine(NSE)
    - Can detect and exploit vulnerabilities

##### Nmap Basics
https://nmap.org/book/

https://www.amazon.com/Nmap-Network-Scanning-Official-Discovery/dp/0979958717

**KEY REMINDER:** Processes run on ports that implement some protocol and that happens at the application level, while firewalls sit in front of those processes and determine if it should let traffic through.

** Port Status **
- **Open:** This indicates that an application is listening for connections on this port.
- **Closed:** This indicates that the probes were received but there is no application listening on this port.
- **Filtered:** This indicates that the probes were not received and the state could not be established. It also indicates that the probes are being dropped by some kind of filtering.
- **Unfiltered:** This indicates that the probes were received but a state could not be established.
- **Open/Filtered:** This indicates that the port was filtered or open but Nmap couldn’t establish the state.
- **Closed/Filtered:** This indicates that the port was filtered or closed but Nmap couldn’t establish the state.

##### Scan Types

** Default Scans **

By default, Nmap performs a SYN Scan, though it substitutes a connect scan if the user does not have proper privileges to send raw packets (requires root access on Unix)

** DEMO: ** Default scan is noisy

1. In Kali, download and install `etherape`
2. Start `etherape`
3. Run a default network scan

- Default Network Scan using CIDR notation: `nmap 192.168.0.1\24`
- Scan Multiple Hosts: `nmap 192.168.0.2 192.168.0.4`
- Scan a Domain: `nmap sudokrew.com`
- Scan a Range of IP Addresses: `nmap 192.168.0.1-192.168.0.100`
- Scan a list of targets: `nmap -iL hosts.txt`
- Scan random hosts on the internet: `nmap -iR 20`
- Exclude certain targets from a scan: 
    - `nmap 192.168.0.2\24 --exclude 192.168.0.5`
    - `nmap 192.168.0.2\24 --exclude-file exclude-hosts.txt`
- Understanding **what** Nmap does! 
    - Wireshark capture
    - `-v` verbose flag

** Basic Scan Types (-sT, -sS) **

- TCP connect() Scan [-sT]
    - Easily detected
    - Gets it's name from using the connect() Unix socket system call to connect to a remote machine
    - Port reported as open is definitely able to be connected to as Nmap did just that
    - IDS will often log a connection request to a port that is not open
- SYN Stealth Scan [-sS]
    - Also known as a half-open scan because the connection is never fully established as in a connect scan
    - TCP stealth scan was developed for the above limitations
    - SYN Stealth scan while *more* stealthy is still detectable
    - TCP scan leverages 3-way handshake protocol specifics
        - Nmap sends TCP packet while modifying TCP header flags
        - Nmap can detect three steps with a Stealth Scan
            - Open
            - Closed
            - Filtered
        - 4 flags we are concerned with for this scan:
            - SYN(Synchronize) - Start communication sequence
                - Include sequence number
            - ACK(Acknowledge) - Acknowledge receipt of packet
            - RST(Reset) - Reset connection, close immediately
            - FIN(Finish) - Communication is done
        - SYN/ACK Flow
            - Host sends SYN packet and sequence number
            - Target responds with SYN/ACK and next sequence number if accepting connections
                - SYN/ACK means that port is open and process is running on port
            - Host system then sends ACK to acknowledge receipt of response
            - Connection is established
        - Nmap sends SYN packet and awaits response:
            - SYN/ACK received, port is open, send RST before connection is established fully
            - If port is closed, RST packet will be sent
            - If port is filtered, packet will be dropped on the floor and no response will be received
            
** Alternative Scan Types (-sF, -sN, -sX) **

Since modern firewalls and IDS are often looking for SYN scans the following are alternative scanning methods for determining information about running services on a target.

- FIN Scan [-sF] - Send a packet with the FIN flag
- Null Scan [-sN] - Send a packet with no flags set
- Xmas Scan [-sX] - Send a packet with the FIN, PUSH, and URG flags set
- Maimon Scan [-sM] - Send a packet with ACK, FIN flags set

All three of these types of scans use other TCP flags to elicit a RST response. A closed port would respond with a RST packet. While an open port would just drop them. 

These scan types will work against any system where the TCP/IP implementation
follows RFC 793. Windows does NOT follow that RFC.

None of these three methods will ever send a SYN packet.

** Ping Scan (-sP, -P0) **

A ping scan is just used to see which hosts are online and responding to a request.

Will send an ICMP echo request first and if no ICMP echo response is recieved, it will try a TCP ping which sends a SYN or ACK to any port to determine if it gets a response back. 

Using the `-P0` flag will disable the ICMP echo request and only try a TCP ping.

** NOTE ** A ping scan can also be used to aquire MAC address of target.

** Protocol Scan (-sO) **
- A protocol scan can determine which protocols are supported on a target.

** UDP Scan (-sU) **

A UDP scan will check for UDP specific services like DNS and DHCP. This type of scan is often overlooked but should be considered.

`nmap -sU 10.0.1.1`

** Version Detection (-sV) **

You can scan for version information about a service with a version detection scan. 

*NOTE:* Using the `-A` option would allow both OS fingerprinting and version detection.

Version scan techniques are quite complex and more information can be discovered here:
[Version Scan Information](https://nmap.org/book/vscan.html)

** Agressive/ACK Scan (-sA) **

To gain more granular information about firewall rules you can use the ACK scan. 

If a port sends back a RST to an ACK packet, the port is reported as "unfiltered" as it likely made it's way through the firewall. 

If nothing comes back, the port is reported as "filtered" as the firewall likely blocked the request from reaching the process.

This kind of scan can also determine if firewalls are stateful or stateless but will never report if a port is "open", so it should be used in conjunction with other scans.

** Idle Scan (-sI) **

Example: `nmap -sI 10.0.1.18:113 -Pn -p20-80,110-180 -r –packet-trace -v 10.0.1.45`

Idle scan makes use of a zombie host to never show the originating host and can be used for man in the middle attacks. This scan has the benefit of hiding the originating source but will also **determine trust relationshps between hosts**.

*How it Works:*
Most machines are configured to use a fragment identification number or IP ID. Operating systems will increment this number for every packet sent. You can query this number to see how many packets have been sent since previous request. 

*Steps to execute*
1. Probe the zombie IP and record the Zombie IP ID.
2. Send a spoofed ICMP packet to the target with the zombie IP as the originating host.
3. The response will be sent back to the zombie since the target thinks that's where the originating request came from
4. Probe the zombie IP again and compare.
    - If the zombie IP ID has incremented by one it hasn't received a response and sent it's own packet back
    - If the zombie IP ID has incremented by two you know it received a response from the target
    - If the zombie IP has incremented by two, those results should not be relied on
    
A good zombie has:
- Low traffic, not generating IP IDs frequently
- Increments IP ID globally, does not keep separate ids per host

An operating system `-O` scan with `-v` will detect the IP ID generation method. We are looking for `Incremental` as the target value.

** NOTE ** `hping` has better functionality for determining a good zombie host


** Window Scan, RPC Scan, List Scan [-sW,
-sR, -sL] **

- Window Scan [-sW]: Similar to an ACK scan but scans the window property on a RST packet for window size to determine if a port is open 
- RPC Scan [-sR]: Used to scan for RPC services usually found on Linux systems that are running the NFS service 
- List Scan [-sL]: Scan a list of ips and do a reverse DNS lookup


** OS Scans **

An OS scan uses a series of packet requests and combinations to determine the operating system and service information about the target. It is additionally configurable with the `--osscan-guess` flag to try to determine a closer match on the target OS.

`nmap -O 192.168.1.17 --osscan-guess`

** NOTE ** In order for the OS scan work perfectly there must be at least one open and one closed port on the target system

It does this through a process called OS fingerprinting. More information can be found here:

- https://nmap.org/book/osdetect-fingerprint-format.html
- https://nmap.org/book/osdetect.html

** Misc. **

- `-A` - Simple flag to use common detection options[OS detection, version detection, script scanning and traceroute]
- `-v` - Verbose mode. Show detailed information about the current steps of the scan
- `nmap 192.158.2.43-192.158.2.55` - Scan range of IP addresses
- `-iL somefile` - Read list of hosts/ip addresses from a file
- `-iR 5` - Scan random number of hosts on the internet
- `-exclude 192.156.23.2` - Exclude certain hosts from a scan
- `-p 22,80,443` - Specify ports to scan
- `-traceroute` - Perform a traceroute to determine network layout
- `-R` - Perform a reverse DNS lookup on all IP addresses
- `-system-dns` - Use the system DNS rather than Nmap internal
- `–dns-servers server1 server2` - Use specific DNS servers
- `-scanflags URGPSHFIN` - Allow customization of flags sent for evasion of firewall IDS from common flag settings(bit or FLAG names with no spacing)
- `-p-` - Scan all 65535 ports


** DNS Bruteforce **

`sudo nmap -sP --script discovery sudokrew.com`

If certain DNS naming conventions are used, we can assume that certain types of services are running on a particular server and start to profile certain vulnerability exploits.

##### Advanced Nmap
- [Understanding an Nmap Fingerprint](https://nmap.org/book/osdetect-fingerprint-format.html)
- [Understanding OS Detection](https://nmap.org/book/osdetect.html)

##### Scanning Steps - Scenario 1
1. Sweep the Network with a ping scan to determine which hosts are available
2. Document those results and take note of MAC and IP
3. Identify targets and probe more directly to determine OS and services running

##### Firewall/IDS Evasion

** RULE ** Don't ping hosts as ICMP requests are often blocked or detected. Use the `-PN scan` instead.

** NOTE ** A host that responds to a ping sweep but reports no OS or open ports or services detected are likely a firewall or IDS.

###### Turning off Ping
The -P0 option can be used with most ping scan options to turn off ICMP ping requests.

"Disabling pings has two advantages: First, it adds extra stealth if you’re
running one of the more stealthy attacks, and secondly it allows Nmap to scan
hosts which don’t reply to pings (ordinarily, Nmap would report those hosts as
being ”down” and not scan them)."

Alternative Scenarios:
`sudo nmap -PA sudokrew.com` Blocks ICMP and fails
`sudo nmap -PA -P0 sudokrew.com` TCP scan succeeds

`sudo nmap sudokrew.com` Doesn't show blackice-icecap
`sudo nmap -PE sudokrew.com` Shows blackice-icecap

`-PP` and `-PM` options use different ICMP registers sometimes getting ICMP echo requests past certain firewalls



###### Idle Scan -sI
The best way to avoid detection is the idle scan with pings disabled `-sI -P0` as no packets will come from your actual IP address to the target.

###### Timing
The slower a scan, the less likely it is an IDS or firewall will detect the pattern of a ping followed by connect requests. A good IDS will detect pings coming from an IP followed by connect requests to a series of ports.

Use the Paranoid(`-T 0`) or Sneaky(`-T 1`) timing flags to evade detection.

** WARNING ** Paranoid defaults to 5 minutes between requests and Sneaky is 15 seconds

Use the `--scan-delay` flag to specify custom times to wait between probes.

If you want fast results at the expense of accuracy you can use the following flags to speed up a scan:
- `-min-parallelism, -max-parallelism` - Set the number of concurrent requests
- `-min-hostgroup, -max-hostgroup` - Set the number of groupings of hosts to scan concurrently
- `-F` - Fast scan

##### Decoys
You can not mask your IP address many times, but you can send requests that appear to come from multiple IP addresses to disguise your actual IP address with the Decoy(`-D xxx.xxx.xxx.xxx, xxx.xxx.xxx.xxx`) flag to make it harder to determine the actual source.

To generate random IPs you can use the `-D RND:10` syntax.

##### Fragmenting
You can specify the `-f` option with the main TCP scan types(`-sS, -sF, -sN, -sX`) to fragment the packets which makes it harder for a firewall or packet filter to determine the contents of a packet.

##### Source Port
You can specify the `–source-port` option to take advantage of loose firewall rules defined. 

** NOTE ** Remember this is to get through the firewall to the host to scan ports further

##### Spoof MAC Address
You can spoof your MAC address with the `–spoof-mac` flag. This could be helpful for MAC filtering if you know one of the allowed hosts MAC address. Adding a value of `0` will generate a random MAC address and specifying a vendor name will generate a random MAC address by that vendor.

#### 2. Pre-requisites

#### 3. Desired Outcomes

#### 4. Resources

- http://resources.infosecinstitute.com/nmap-cheat-sheet/#article
- http://resources.infosecinstitute.com/nmap-cheat-sheet-discovery-exploits-part-2-advance-port-scanning-nmap-custom-idle-scan/#article
- http://resources.infosecinstitute.com/nmap-cheat-sheet-discovery-exploits-part-3-gathering-additional-information-host-network-2/#article
- http://resources.infosecinstitute.com/nmap-cheat-sheet-part-4/#article
- http://resources.infosecinstitute.com/nmap-cheat-sheet-5-the-final-view-of-a-ninja-pentester/#article
- https://null-byte.wonderhowto.com/how-to/hack-like-pro-advanced-nmap-for-reconnaissance-0151619/
- http://www.csc.villanova.edu/~nadi/csc8580/S11/nmap-tutorial.pdf
- https://nmap.org/book/man-port-scanning-techniques.html
- https://nmap.org/book/osdetect-fingerprint-format.html
- https://nmap.org/book/osdetect.html
- https://en.wikipedia.org/wiki/Idle_scan

#### 5. Challenges/Exercises
- Setup a backdoor on a system and use Nmap to discover
- Install fail2ban and scan that host and see how you can configure to get even stealthier requests to jail
- Look at firewall logs for the scans you are creating and see what makes it's way to the logs and what doesn't

#### 6. Self Study Objectives

#### 7. Estimated Time

#### 8. Applicable Standards
